In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('missing').getOrCreate()

In [10]:
df_pyspark=spark.read.csv('sample2.csv',header=True,inferSchema=True)

In [11]:
df_pyspark

DataFrame[Name: string, age: int, Experience: int, Salary: int]

In [12]:

df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [13]:

df_pyspark.show()

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
|sourav|  31|        10| 30000|
| sunil|  30|      NULL| 25000|
| soyel|  29|         4| 20000|
|souvik|  24|         3| 20000|
|apurba|NULL|         1| 15000|
|    pk|  23|         2|  NULL|
|  NULL|  23|         2| 34000|
+------+----+----------+------+



In [14]:
##drop the columns
df_pyspark.drop('Name').show()

+----+----------+------+
| age|Experience|Salary|
+----+----------+------+
|  31|        10| 30000|
|  30|      NULL| 25000|
|  29|         4| 20000|
|  24|         3| 20000|
|NULL|         1| 15000|
|  23|         2|  NULL|
|  23|         2| 34000|
+----+----------+------+



In [15]:

df_pyspark.show()

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
|sourav|  31|        10| 30000|
| sunil|  30|      NULL| 25000|
| soyel|  29|         4| 20000|
|souvik|  24|         3| 20000|
|apurba|NULL|         1| 15000|
|    pk|  23|         2|  NULL|
|  NULL|  23|         2| 34000|
+------+----+----------+------+



drop rows containing null values

In [16]:
df_pyspark.na.drop().show()

+------+---+----------+------+
|  Name|age|Experience|Salary|
+------+---+----------+------+
|sourav| 31|        10| 30000|
| soyel| 29|         4| 20000|
|souvik| 24|         3| 20000|
+------+---+----------+------+



how="any": This parameter specifies that any row containing at least one null value should be dropped. Other options include "all", which would drop rows only if all values are null.

In [17]:
### any==how
df_pyspark.na.drop(how="any").show()

+------+---+----------+------+
|  Name|age|Experience|Salary|
+------+---+----------+------+
|sourav| 31|        10| 30000|
| soyel| 29|         4| 20000|
|souvik| 24|         3| 20000|
+------+---+----------+------+



drop rows containing atleast 3 null values

In [18]:
##threshold
df_pyspark.na.drop(how="any",thresh=3).show()

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
|sourav|  31|        10| 30000|
| sunil|  30|      NULL| 25000|
| soyel|  29|         4| 20000|
|souvik|  24|         3| 20000|
|apurba|NULL|         1| 15000|
|    pk|  23|         2|  NULL|
|  NULL|  23|         2| 34000|
+------+----+----------+------+



drop rows in which age column contains null

In [19]:
##Subset
df_pyspark.na.drop(how="any",subset=['Age']).show()

+------+---+----------+------+
|  Name|age|Experience|Salary|
+------+---+----------+------+
|sourav| 31|        10| 30000|
| sunil| 30|      NULL| 25000|
| soyel| 29|         4| 20000|
|souvik| 24|         3| 20000|
|    pk| 23|         2|  NULL|
|  NULL| 23|         2| 34000|
+------+---+----------+------+



In [23]:
### Filling the Missing Value
df_pyspark.na.fill(value='Missing Values',subset=['Name']).show()

+--------------+----+----------+------+
|          Name| age|Experience|Salary|
+--------------+----+----------+------+
|        sourav|  31|        10| 30000|
|         sunil|  30|      NULL| 25000|
|         soyel|  29|         4| 20000|
|        souvik|  24|         3| 20000|
|        apurba|NULL|         1| 15000|
|            pk|  23|         2|  NULL|
|Missing Values|  23|         2| 34000|
+--------------+----+----------+------+



In [26]:
df_pyspark.show()

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
|sourav|  31|        10| 30000|
| sunil|  30|      NULL| 25000|
| soyel|  29|         4| 20000|
|souvik|  24|         3| 20000|
|apurba|NULL|         1| 15000|
|    pk|  23|         2|  NULL|
|  NULL|  23|         2| 34000|
+------+----+----------+------+



In [27]:
from pyspark.ml.feature import Imputer

In [28]:
num_imputer = Imputer(
    inputCols=['age', 'Experience', 'Salary'], 
    outputCols=["{}_imputed".format(c) for c in ['age', 'Experience', 'Salary']]
    ).setStrategy("median")

In [30]:
num_imputer.fit(df_pyspark).transform(df_pyspark).show()

+------+----+----------+------+-----------+------------------+--------------+
|  Name| age|Experience|Salary|age_imputed|Experience_imputed|Salary_imputed|
+------+----+----------+------+-----------+------------------+--------------+
|sourav|  31|        10| 30000|         31|                10|         30000|
| sunil|  30|      NULL| 25000|         30|                 2|         25000|
| soyel|  29|         4| 20000|         29|                 4|         20000|
|souvik|  24|         3| 20000|         24|                 3|         20000|
|apurba|NULL|         1| 15000|         24|                 1|         15000|
|    pk|  23|         2|  NULL|         23|                 2|         20000|
|  NULL|  23|         2| 34000|         23|                 2|         34000|
+------+----+----------+------+-----------+------------------+--------------+



In [39]:
obj_inputer = Imputer(
    inputCols=['Name'], 
    outputCols=["{}_imputed".format(c) for c in ['Name']]
    ).setStrategy("most_frequent")

In [40]:
obj_inputer.fit(df_pyspark).transform(df_pyspark).show()

IllegalArgumentException: Imputer_af315aab1247 parameter strategy given invalid value most_frequent.